In [1]:
from math import log, sqrt, pi, exp
from scipy.stats import norm
from scipy.optimize import minimize_scalar
from datetime import datetime, date
import numpy as np
import pandas as pd
import pandas_datareader.data as web
from pandas import DataFrame
import pickle

## Black-Scholes Model
Implementation of the Black-Scholes Model for the price calculation of european Call/Put options.
<br>
<br>
__Symbols:__
<br>
S = Current Stock Price
<br>
K = Strike Price
<br>
t = time to maturity
<br>
r = risk-free rate (US Treasury Bonds, because of missing API support)
<br>
sigma = volatility

In [2]:
# calculation
def d1(S,K,T,r,sigma):
    return(log(S/K)+(r+sigma**2/2.)*T)/(sigma*sqrt(T))
def d2(S,K,T,r,sigma):
    return d1(S,K,T,r,sigma)-sigma*sqrt(T)

In [4]:
# european call
def bs_call(S,K,T,r,sigma):
    return S*norm.cdf(d1(S,K,T,r,sigma))-K*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma))

# european put
def bs_put(S,K,T,r,sigma):
    return K*exp(-r*T)-S+bs_call(S,K,T,r,sigma)

## Getting Data via Alpha Vantage

In [5]:
today = date.today()
one_year_ago = today.replace(year=today.year-1)

today: 24
last year: 2021-05-24


### getting company data

In [6]:
# just 4 companys due to API restrictions
dax_40 = pickle.load(open("stock_timeseries_dict.p", "rb"))
dax_40["ADS.DE"]

,open,high,low,close,volume
2021-05-25,304.40,304.40,299.15,299.15,555295
2021-05-26,299.00,299.35,294.45,297.05,515046
2021-05-27,295.50,298.35,293.35,297.25,578908
2021-05-28,298.60,299.90,295.10,299.35,315651
2021-05-31,299.00,302.90,297.65,298.35,279491
...,...,...,...,...,...
2022-05-16,179.46,181.68,177.82,180.06,648134
2022-05-17,181.90,186.82,180.44,183.30,630018
2022-05-18,183.40,184.08,175.14,175.98,782072
2022-05-19,175.98,175.98,175.98,175.98,730987


In [7]:
for k in dax_40.keys():
    dax_40[k] = dax_40[k].sort_index(ascending = False)
    dax_40[k] = dax_40[k].dropna()
    dax_40[k] = dax_40[k].assign(close_day_before=dax_40[k].close.shift(-1))
    dax_40[k]['returns'] = ((dax_40[k].close - dax_40[k].close_day_before)/dax_40[k].close_day_before)
dax_40["ADS.DE"]

,open,high,low,close,volume,close_day_before,returns
2022-05-20,170.24,175.34,168.30,172.80,1035957,175.98,-0.018070
2022-05-19,175.98,175.98,175.98,175.98,730987,175.98,0.000000
2022-05-18,183.40,184.08,175.14,175.98,782072,183.30,-0.039935
2022-05-17,181.90,186.82,180.44,183.30,630018,180.06,0.017994
2022-05-16,179.46,181.68,177.82,180.06,648134,180.48,-0.002327
...,...,...,...,...,...,...,...
2021-05-31,299.00,302.90,297.65,298.35,279491,299.35,-0.003341
2021-05-28,298.60,299.90,295.10,299.35,315651,297.25,0.007065
2021-05-27,295.50,298.35,293.35,297.25,578908,297.05,0.000673
2021-05-26,299.00,299.35,294.45,297.05,515046,299.15,-0.007020


## Calculating Option Price

### setting up parameters for Calculation

In [8]:
df_risk_free = pickle.load(open("risk_free_rate.p", "rb"))
df_risk_free = df_risk_free[["date", "value"]].copy()
df_risk_free = df_risk_free.sort_index(ascending = False)
df_risk_free
#uty = (web.DataReader("^TNX", 'yahoo', today.replace(day=today.day-3), today)['Close'].iloc[-1])/100

,date,value
27,2022-04-01,0.007500
26,2022-03-01,0.002800
25,2022-02-01,0.001500
24,2022-01-01,-0.001148
23,2021-12-01,-0.003843
22,2021-11-01,-0.003136
21,2021-10-01,-0.002043
20,2021-09-01,-0.003627
19,2021-08-01,-0.005386
18,2021-07-01,-0.004514


In [9]:
stock = 'BAS.DE' # Company ticker
expiry = '2022-12-14'
strike_price = 59
relation = 0.1 #Bezugsverhältnis

In [10]:
# just execute, no changes
df = dax_40[stock]

sigma = np.sqrt(252) * df['returns'].std()
lcp = df['close'].iloc[1]
t = (datetime.strptime(expiry, "%Y-%m-%d") - datetime.utcnow()).days / 365
uty = df_risk_free["value"].iloc[1]


print("+++ " + str(stock) + " +++")
print('The Put Option Price is: ', bs_put(lcp, strike_price, t, uty, sigma)*relation)
print('The Call Option Price is: ', bs_call(lcp, strike_price, t, uty, sigma)*relation)

+++ BAS.DE +++
The Put Option Price is:  1.1217146907800863
The Call Option Price is:  0.11689537618288277


### Alles hier nach noch nicht fertig

## implied volatility

In [11]:
def implied_vol(opt_value, S,K,T,r,type_='call'):
    
    def call_obj(sigma):
        return abs(bs_call(S,K,T,r,sigma) - opt_value)
    
    def put_obj(sigma):
        return abs(bs_put(S,K,T,r,sigma) - opt_value)
    
    if type_ == 'call':
        res = minimize_scalar(call_obj, bounds=(0.01,6), method='bounded')
        return res.x
    elif type_ == 'put':
        res = minimize_scalar(put_obj, bounds=(0.01,6),
                              method='bounded')
        return res.x
    else:
        raise ValueError("type_ must be 'put' or 'call'")

In [14]:
C = bs_call(lcp, strike_price, t, uty, sigma)
iv = implied_vol(C,lcp,strike_price,t,uty)
print(iv)
print(sigma)

0.28361219679799693
0.28361285363237027
